In [296]:
import yfinance as yf
import scipy
from scipy import linalg as SLA
from scipy import optimize as opt
import numpy as np
from numpy import linalg as LA
import cvxopt as cv
import mosek
from mosek import fusion
from mosek.fusion import *
import mosek.fusion.pythonic 
import pandas as pd
import sys
import matplotlib.pyplot as plt

In [326]:
#Top 10 most active tech stocks in the US
tickers = ['MSFT','ORCL','ADBE','PANW','PLTR','SNPS','CRWD','FTNT','SQ','NET']

#convert raw data into a pandas dataframe object and select closing price returns
raw = yf.download(tickers,period='5y',interval='1wk')
data = pd.DataFrame(raw)
rets = np.log(data/data.shift(1))
rets = rets.dropna()
rets = rets.Close

covM = np.asarray(rets.cov() * 260)
mu = np.asarray(rets.mean() * 260)

[*********************100%***********************]  10 of 10 completed


In [327]:
#Calculate covariance matrix of returns
noa = len(tickers)
weights = np.random.random(noa)
weights /= np.sum(weights)

#Define functions we want to optimize for
def port_ret(weights):
    return np.sum(weights * rets.mean() * 260)
def port_vol(weights):
    return np.dot(weights.T,np.dot(covM,weights))

In [348]:
#We want our portfolio subject to Ax=B linear constraints
# Define function solving the optimization model
def modifiedMarkowitz(N, m, G, gammas, slack):
    with Model("markowitz") as M:
        riskgrad = []
        port = []
        slack = []
        # Settings
        #M.setLogHandler(sys.stdout) 

        # Decision variable (fraction of holdings in each security)
        # The variable x is restricted to be positive, which imposes the constraint of no short-selling.   
        x = M.variable("x", N, Domain.greaterThan(0.0))
        if slack:
            s = M.variable("slack", Domain.unbounded())
            M.constraint('budget', Expr.sum(x) == 1 + s)
            gamma = M.parameter()
            for g in gammas:
                gamma.setValue(g)
            # Solve optimization
                M.solve()
            # Check if the solution is an optimal point
                solsta = M.getPrimalSolutionStatus()
                if (solsta != SolutionStatus.Optimal):
                    # See https://docs.mosek.com/latest/pythonfusion/accessing-solution.html about handling solution statuses.
                    raise Exception("Unexpected solution status!") 
                returns = M.primalObjValue()
                riskgrad.append(returns)
                port.append(x.level())
                slack.append(s.level())
            return riskgrad,port,slack

        M.constraint('budget', Expr.sum(x) == 1)
        # Objective 
        M.objective('obj', ObjectiveSense.Maximize, x.T @ m)

        # Imposes a bound on the risk
        M.constraint('risk', Expr.vstack(gammas, 0.5, G.T @ x), Domain.inRotatedQCone())
        solsta = M.getPrimalSolutionStatus()
        if (solsta != SolutionStatus.Optimal):
        # See https://docs.mosek.com/latest/pythonfusion/accessing-solution.html about handling solution statuses.
            raise Exception("Unexpected solution status!") 
        returns = M.primalObjValue()
        portfolio = x.level()

    return portfolio,returns

In [353]:
G = SLA.cholesky(covM)
#Convert variables into Matrix fusion objects
gammas = np.linspace(0.01,0.5,30)
alphas,ports,slacks = modifiedMarkowitz(noa,mu,G,gammas,slacks=True)

plt.plot(gammas,alphas,'.')
plt.xlabel('variance')
plt.ylabel('portfolio return')

TypeError: modifiedMarkowitz() got an unexpected keyword argument 'slacks'

In [354]:
mu = np.array([0.07197349, 0.15518171, 0.17535435, 0.0898094 , 0.42895777, 0.39291844, 0.32170722, 0.18378628])
Sigma = np.array([
        [0.09460323, 0.03735969, 0.03488376, 0.03483838, 0.05420885, 0.03682539, 0.03209623, 0.03271886],
        [0.03735969, 0.07746293, 0.03868215, 0.03670678, 0.03816653, 0.03634422, 0.0356449 , 0.03422235],
        [0.03488376, 0.03868215, 0.06241065, 0.03364444, 0.03949475, 0.03690811, 0.03383847, 0.02433733],
        [0.03483838, 0.03670678, 0.03364444, 0.06824955, 0.04017978, 0.03348263, 0.04360484, 0.03713009],
        [0.05420885, 0.03816653, 0.03949475, 0.04017978, 0.17243352, 0.07886889, 0.06999607, 0.05010711],
        [0.03682539, 0.03634422, 0.03690811, 0.03348263, 0.07886889, 0.09093307, 0.05364518, 0.04489357],
        [0.03209623, 0.0356449 , 0.03383847, 0.04360484, 0.06999607, 0.05364518, 0.09649728, 0.04419974],
        [0.03271886, 0.03422235, 0.02433733, 0.03713009, 0.05010711, 0.04489357, 0.04419974, 0.08159633]
      ])
N = mu.shape[0]
gamma2 = 0.05
G = SLA.cholesky(Sigma)

mu2 = np.asarray(rets.mean() * 260)
Sigma2 = np.asarray(rets.cov() * 260)
N2 = mu2.shape[0]
G2 = SLA.cholesky(Sigma2)

In [385]:
f,x = modifiedMarkowitz(N,mu,G,gamma2,slack=False)

Exception: Unexpected solution status!

In [387]:
f,x,y = modifiedMarkowitz(N,mu,G,gamma2,slack=False)

ValueError: Invalid argument list vstack(array([0.01      , 0.02689655, 0.0437931 , 0.06068966, 0.07758621,
       0.09448276, 0.11137931, 0.12827586, 0.14517241, 0.16206897,
       0.17896552, 0.19586207, 0.21275862, 0.22965517, 0.24655172,
       0.26344828, 0.28034483, 0.29724138, 0.31413793, 0.33103448,
       0.34793103, 0.36482759, 0.38172414, 0.39862069, 0.41551724,
       0.43241379, 0.44931034, 0.4662069 , 0.48310345, 0.5       ]),0.5,mosek.fusion.ExprReshape). Candidates are
	mosek.fusion.Expr.vstack(array(mosek.fusion.Expression,ndim=1))
	mosek.fusion.Expr.vstack(mosek.fusion.Expression,double)
	mosek.fusion.Expr.vstack(double,mosek.fusion.Expression)
	mosek.fusion.Expr.vstack(mosek.fusion.Expression,mosek.fusion.Expression)
	mosek.fusion.Expr.vstack(double,mosek.fusion.Expression,mosek.fusion.Expression)
	mosek.fusion.Expr.vstack(mosek.fusion.Expression,double,mosek.fusion.Expression)
	mosek.fusion.Expr.vstack(double,mosek.fusion.Expression,double)
	mosek.fusion.Expr.vstack